# Parallel Computation of $\pi$

This is a short exercise in performance modeling, based on what we saw in the introductory slides.

Due: **9:30 am (before class), Tuesday, August 27.**  Submit your answers to canvas.  You can submit them either as a pdf, or by inserting your answers as cells into this notebook and submitting the notebook.

When we want to compare two algorithms $f$ and $g$ that do the same thing, we look at their runtimes, which will be functions
with parameters, such as the problem size $N$ and the number of processes working together $P$.

Sometimes we will find that one algorithm dominates the other: that $T_f(N,P) < T_g(N,P)$ for all $N$ and all $P$.  
Most of the time, however, neither algorithm is dominant, and we start to ask questions, like "when (for what values of $N$ and $P$) is $T_f$ less than $T_g$?"  This exercise tries to familiarize you with that sort of thinking.

---

Suppose I care about computing $N$ digits of $\pi$, and I have two pseudocode algorithms available.

The first (roughly corresponding to [Borwein's quadratic algorithm](https://en.wikipedia.org/wiki/Borwein%27s_algorithm#Quadratic_convergence_(1984)) looks like the following:

In [ ]:
from numpy import log2

def f(N):
    a, b, p = A()
    for i in range(int(log2(N))):
        a_new = B(a,b)
        b_new = C(a,b)
        p_new = D(a_new, b_new, p)
        a, b, p = (a_new, b_new, p_new)
    return p

Suppose that the only things that take time in this algorithm are the functions `A`, `B`, `C`, and `D`, and that $T_A = 11$, $T_B = 21$, $T_C = 13$ and $T_D = 13$.

**1 (1 pt).** What is $T_f(N, 1)$, the time it takes one process to run `f`?

**2.** Suppose that multiple threads can divide up the operations in `f` and work in parallel without interfering with each other (but of course an operation can't begin until its inputs are available).

  **a (1 pt).** What is the degree of concurrency in `f` (how many processes can collaborate on `f` before there is no more work to do)?

**Hint**: Assuming that `A`, `B`, `C`, `D` cannot be shared, we want to find the largest set of operations that can be assigned independently, meaning that the inputs of one operation are not the outputs of another.  If we look just within the loop body without unrolling, we can see that `B` and `C` are independent, so a concurrency of two is acceptable for this assignment (see the attached image).

 ![graph](./graph.jpg)

**however**, some of your fellow students noticed that when you unroll this loop (draw multiple copies of the loop body), you can see more concurrency.  If you are looking to resubmit this assignment, you should try to find that additional concurrency.

  **b (1 pt)**. Using this model, what is the smallest value of $T_f(N,P)$ over all $P$?

**Hint:** This is asking you to compute $T_f(N,P)$ when $P$ is equal to the maximum concurrency.  The time taken by a computation graph when items can happen concurrently is the [critical path](https://en.wikipedia.org/wiki/Critical_path_method).  Assign a value to each node in the graph equal to its runtime, and find the path (chain of dependendent operations) from the inputs to outputs that has the largest sum.

Our algorithm will still have a loop structure, so you only need to find the time taken before entering the first loop, the time in the loop, and the time after the last loop.  The time taken by each of these portions should be easy to compute.

The next algorithm (roughly corresponding to [Gregory's series](https://en.wikipedia.org/wiki/Gregory%27s_series)) looks like this:

In [ ]:
def g(N):
    p = 0.
    for i in range(N**2):
        p = p + E(i)
    return p

Suppose that the only thing that takes time in this algorithm are $E$ and addition, and that $T_E = 10$ and $T_+ = 1$.

**3 (1 pt).** What is $T_g(N,1)$, the time it takes one process to run `g`?

**4.** With the same rules of parallelism as before,

  **a (1 pt).** What is the degree of concurrency in `g`? (Hint: assume addition is associative, that a sum like $a + b + c + d + ...$ can be done in any order)

**Hint:** Don't let the fact that the loop iterations occur one after the other in a sequential implementation confuse you: the same rule applies.  We are looking for the maximum number of *independent* nodes in the computation graph, none of which needs inputs from the outputs of the others.

Consider that the following two graphs *are the same graph*:

![graph2](./graph2.jpg)

  **b (1 pt).** Suppose that $P$ processes, working together, can sum a list of $P$ numbers together in $T_{sum}(P) = \log_2 P$ time.  Using this operation, what is a simple model for $T_g(N,P)$?

**Hint:** I recommend giving each process either $\lceil N^2 / P \rceil$ or $\lfloor N^2 / P \rfloor$ iterations of the loop to work on, and having each process compute the *partial sum* $p_i$ for the iterations it was assigned.  Then compute $p = \sum_{i=1}^P p_i$.  In the first phase of the algorithm, the processes are working independently, so the time for that phase is the maximum time it takes for any process to compute its partial sum.  The time for the second phase is the time I gave you above.

  **c (1 pt).** Using this model, what is the smallest value of $T_g(N,P)$ over all $P$?

**Hint:** For this problem, I recommend assuming $\lceil N^2 / P \rceil = \lfloor N^2 / P \rfloor = N^2 / P$ for simplicity.  Take the derivative with respect to $P$ of $T_g(N,P)$ you computed above to estimate the value $P_{\min}$.  *If* $P_{\min}$ is less than the maximum concurrency, evaluate $T_g(N,P_{\min})$, otherwise evaluate $T_g(N,P)$ for $P$ equal to the maximum concurrency.

**5 (1 pt).** Suppose we want to compute 16 digits of $\pi$: using your models from (1) and (4) how many processes $P$ do we need before $T_g(16,P) < T_f(16,1)$?

**(Bonus 1 pt)**. Generalize (5) and define a function $P(N)$ that estimates how many processes $P$ are needed before
$T_g(N,P(N)) < T_f(N,1)$.